
# Pipeline from Landing to Bronze

Just read `.csv` files and transform them into delta tables on bronze layer.



## Required Libraries


In [0]:
from pathlib import Path
from datetime import datetime

from pyspark.sql.functions import regexp_extract, col, lit
from pyspark.sql.connect.dataframe import DataFrame
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType

from src.common import (
    get_tz,
    add_reference_date_parameters,
    get_reference_dates,
    read_sql_template,
    table_exists, 
    use_schema_and_create_if_not_exists,
)


## Define Unity Catalog

In [0]:
CATALOG = "precos_pmc"
SCHEMA = "bronze"
tz = get_tz()

In [0]:

add_reference_date_parameters(dbutils.widgets)
use_schema_and_create_if_not_exists(spark, catalog=CATALOG, schema=SCHEMA)



## Collect Dates




In [0]:
dates = get_reference_dates(dbutils.widgets)
print('Running for:', [dt.strftime('%Y-%m-%d') for dt in dates])

## Full Database

In [0]:

def get_full_db_dataframe_from_s3_landing(
    date_reference: datetime
) -> DataFrame:

    """
        Read Full Database file from S3
        
        It was used before of 2023-07-18

    """

    filepattern = 'Clique_Economia_-_Base_de_Dados.csv'

    paths = (
        "s3a://precos-pmc-landing"
        + f"/{date_reference.strftime('%Y')}" # YYYY
        + f"/{date_reference.strftime('%m')}" # MM
        + f"/{date_reference.strftime('%d')}" # DD
        + f"/{date_reference.strftime('%Y-%m-%d')}_{filepattern}"
    )

    try:
        
        # Lazy
        df = (
            spark.read

                # Read all
                .option("header", "true")
                .option("inferSchema", "true")
                .csv(paths)
                
                # Extract filename
                .withColumn("file_path", col("_metadata.file_path"))
                .withColumn(
                    "filename",
                    regexp_extract(col("_metadata.file_path"), r"([^/]+$)", 1)
                )
                .drop("file_path")
        )

        # Materialize
        df.count()

        # Return materialized
        return df

    except AnalysisException as e:

        if "PATH_NOT_FOUND" in e.getMessage():
            print(f"Landing file not found (expected): {paths}")
            return spark.createDataFrame([], StructType([]))
        else:
            # Anything else is a real failure
            raise
        
    except Exception as e:
        print(f'Error reading {filepattern}: {e}')
        return spark.createDataFrame([], StructType([]))





## Daily database

After 2023-07-18, the datasets were daily snapshots, and no longer the full database. 



In [0]:

def get_incremental_db_dataframe_from_s3_landing(
    date_reference: datetime
) -> DataFrame:

    """
        Read Incremental Database file from S3
        
        It is being used after of 2023-07-18

    """

    filepattern = 'Clique_Economia_-_Cotacoes_-_Base_de_Dados.csv'

    paths = (
        "s3a://precos-pmc-landing"
        + f"/{date_reference.strftime('%Y')}" # YYYY
        + f"/{date_reference.strftime('%m')}" # MM
        + f"/{date_reference.strftime('%d')}" # DD
        + f"/{date_reference.strftime('%Y-%m-%d')}_{filepattern}"
    )

    try:
        df = (
            spark.read

                # Read all
                .option("header", "true")
                .option("inferSchema", "true")
                .option("delimiter", ";")
                .option("encoding", "ISO-8859-1")
                .csv(paths)
                
                # Extract filename
                .withColumn("file_path", col("_metadata.file_path"))
                .withColumn(
                    "filename",
                    regexp_extract(col("_metadata.file_path"), r"([^/]+$)", 1)
                )
                .drop("file_path")
        )

        # Materialize
        df.count()

        # Return materialized
        return df

    except AnalysisException as e:

        if "PATH_NOT_FOUND" in e.getMessage():
            print(f"Landing file not found (expected): {paths}")
            return spark.createDataFrame([], StructType([]))
        else:
            # Anything else is a real failure
            raise
        
    except Exception as e:
        print(f'Error reading {filepattern}: {e}')
        return spark.createDataFrame([], StructType([]))



## Products database

In [0]:

def get_product_db_dataframe_from_s3_landing(
    date_reference: datetime
) -> DataFrame:

    """
        Read Product Database file from S3
        
        It is being used before 2023-07-18

    """

    filepattern = 'Clique_Economia_-_Produto_-_Base_de_Dados.csv'

    paths = (
        "s3a://precos-pmc-landing"
        + f"/{date_reference.strftime('%Y')}" # YYYY
        + f"/{date_reference.strftime('%m')}" # MM
        + f"/{date_reference.strftime('%d')}" # DD
        + f"/{date_reference.strftime('%Y-%m-%d')}_{filepattern}"
    )

    try:
        df = (
            spark.read

                # Read all
                .option("header", "true")
                .option("inferSchema", "true")
                .csv(paths)
                
                # Extract filename
                .withColumn("file_path", col("_metadata.file_path"))
                .withColumn(
                    "filename",
                    regexp_extract(col("_metadata.file_path"), r"([^/]+$)", 1)
                )
                .drop("file_path")
        )

        # Materialize
        df.count()

        # Return materialized
        return df

    except AnalysisException as e:

        if "PATH_NOT_FOUND" in e.getMessage():
            print(f"Landing file not found (expected): {paths}")
            return spark.createDataFrame([], StructType([]))
        else:
            # Anything else is a real failure
            raise
        
    except Exception as e:
        print(f'Error reading {filepattern}: {e}')
        return spark.createDataFrame([], StructType([]))



## Register on bronze catalog

In [0]:

def table_exists(catalog: str, schema: str, table: str) -> bool:
    return (
        spark.sql(f"SHOW TABLES IN {catalog}.{schema}")
             .filter(f"tableName = '{table}'")
             .count() > 0
    )

def write_bronze_delta_table_on_s3(
    df: DataFrame, 
    date_reference: datetime, 
    table_name: str
) -> bool:

    if df.isEmpty():
        print(f"Nothing to insert in {table_name}")
        return False

    s3_path = (
        "s3a://precos-pmc-bronze"
        f"/{table_name}"
    )

    (y, m, d) = (
        date_reference.year,
        date_reference.month,
        date_reference.day
    )

    full_table_name = f"{CATALOG}.{SCHEMA}.{table_name}"

    df_with_partitions = (
        df
        .withColumn("year", lit(y))
        .withColumn("month", lit(m))
        .withColumn("day", lit(d))
    )

    try:
        
        # Create table
        if not table_exists(CATALOG, SCHEMA, table_name):
            print(f"Creating table {full_table_name}")
            (
                df_with_partitions
                .writeTo(full_table_name)
                .using("delta")
                .partitionedBy("year", "month", "day")
                .create()
            )
    
        # Grants idempotency replaces
        else:

            print(f"Replacing partition {y}-{m}-{d} in {full_table_name}")

            spark.sql(f"""
                DELETE FROM {full_table_name}
                WHERE year = {y} AND month = {m} AND day = {d}
            """)
                    
            
            (
                df_with_partitions
                .writeTo(full_table_name)
                .append()
            )

        print(f'Sucessful inserted into {full_table_name}')
        return True
    
    except Exception as e:
        print(f'Error while writing bronze delta ({table_name}): {e}')
        return False



## Execution flow

In [0]:

for date in dates:
    
  df_product = get_product_db_dataframe_from_s3_landing(date_reference=date)
  df_product = write_bronze_delta_table_on_s3(df=df_product, date_reference=date, table_name='produtos')

  df_incremental_db = get_full_db_dataframe_from_s3_landing(date_reference=date)
  df_incremental_db = write_bronze_delta_table_on_s3(df=df_incremental_db, date_reference=date, table_name='base_incremental')

  df_day = get_incremental_db_dataframe_from_s3_landing(date_reference=date)
  df_day = write_bronze_delta_table_on_s3(df=df_day, date_reference=date, table_name='cotacoes')
